# MCNN的特征图和卷积核的可视化


## 加载模型
MCNN的test文件里面有加载模型的代码,直接使用

In [1]:
import os
import torch
import numpy as np
import sys
import torch.nn as nn

from src.crowd_count import CrowdCounter
from src import network
from src.data_loader import ImageDataLoader
from src.timer import Timer
from src import utils
from src.evaluate_model import evaluate_model

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
vis = False
save_output = True

#使用mcnn的test文件的代码加载mcnn模型


data_path =  './data/original/shanghaitech/part_A_final/test_data/images/'
gt_path = './data/original/shanghaitech/part_A_final/test_data/ground_truth_csv/'
model_path = './saved_models/mcnn_shtechA_660.h5'



output_dir = './output/'
model_name = os.path.basename(model_path).split('.')[0]
file_results = os.path.join(output_dir,'results_' + model_name + '_.txt')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
output_dir = os.path.join(output_dir, 'density_maps_' + model_name)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)


net = CrowdCounter()
      
trained_model = os.path.join(model_path)
network.load_net(trained_model, net)
net.cuda()
net.eval()
mae = 0.0
mse = 0.0

加载的模型存在net中,是CrowdCounter()的类型
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/1.png)
从上图我们看到net.DME存的是MCNN(),因此将net.DME打印出来
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/2.png)
以Conv2d(1, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))为例,输入通道为1,输出通道为16,卷积核个数与输出通道相等即为16,卷积核大小为9*9,池化大小为4*4

In [2]:
net.DME #打印出mcnn神经网络的结构

MCNN(
  (branch1): Sequential(
    (0): Conv2d(
      (conv): Conv2d(1, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (relu): ReLU(inplace)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(
      (conv): Conv2d(16, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(
      (conv): Conv2d(32, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
    (5): Conv2d(
      (conv): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
  )
  (branch2): Sequential(
    (0): Conv2d(
      (conv): Conv2d(1, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): ReLU(inplace)
    )
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(
      (conv): Conv2

由net的load_net文件我们可以知道模型的参数存在state_dict()中,先打印出来看看
![](https://raw.githubusercontent.com/iicanfly/screenshot/v-mcnn/3.png)

In [3]:
params=net.state_dict()   #将net的信息存入params中
for k,v in params.items():
    print(k)    #打印网络中的变量名

DME.branch1.0.conv.weight
DME.branch1.0.conv.bias
DME.branch1.2.conv.weight
DME.branch1.2.conv.bias
DME.branch1.4.conv.weight
DME.branch1.4.conv.bias
DME.branch1.5.conv.weight
DME.branch1.5.conv.bias
DME.branch2.0.conv.weight
DME.branch2.0.conv.bias
DME.branch2.2.conv.weight
DME.branch2.2.conv.bias
DME.branch2.4.conv.weight
DME.branch2.4.conv.bias
DME.branch2.5.conv.weight
DME.branch2.5.conv.bias
DME.branch3.0.conv.weight
DME.branch3.0.conv.bias
DME.branch3.2.conv.weight
DME.branch3.2.conv.bias
DME.branch3.4.conv.weight
DME.branch3.4.conv.bias
DME.branch3.5.conv.weight
DME.branch3.5.conv.bias
DME.fuse.0.conv.weight
DME.fuse.0.conv.bias


In [7]:
bia0 = params['DME.branch1.0.conv.bias']     #将branch1.0中的偏置参数存入bia0中
weigh0 = params['DME.branch1.0.conv.weight']     #将branch1.0中的卷积核存入weigh0中

bia2 = params['DME.branch1.2.conv.bias']     #将branch1.2中的偏置参数存入bia中
weigh2 = params['DME.branch1.2.conv.weight']     #将branch1.2中的卷积核存入weigh中

bia4 = params['DME.branch1.4.conv.bias']     #将branch1.4中的偏置参数存入bia中
weigh4 = params['DME.branch1.4.conv.weight']     #将branch1.4中的卷积核存入weigh中

bia5 = params['DME.branch1.5.conv.bias']     #将branch1.5中的偏置参数存入bia中
weigh5 = params['DME.branch1.5.conv.weight']     #将branch1.5中的卷积核存入weigh中


In [8]:
WW0 = weigh0.cpu().numpy();    #将torch的tensor转化为numpy的数组类型
BB0 = bia0.cpu().numpy()

WW2 = weigh2.cpu().numpy();    #将torch的tensor转化为numpy的数组类型
BB2 = bia2.cpu().numpy()

WW4 = weigh4.cpu().numpy();    #将torch的tensor转化为numpy的数组类型
BB4 = bia4.cpu().numpy()

WW5 = weigh5.cpu().numpy();    #将torch的tensor转化为numpy的数组类型
BB5 = bia5.cpu().numpy()

将DME.branch1.0.conv的卷积核信息输出,有16个单通道卷积核,大小为9*9

In [9]:
print(WW0.shape)  
print(WW2.shape) 
print(WW4.shape) 
print(WW5.shape) 
 

(16, 1, 9, 9)
(32, 16, 7, 7)
(16, 32, 7, 7)
(8, 16, 7, 7)


In [30]:
#print(WW2[0,0])  
#print(WW2)

[[[[ 1.76333804e-02  2.01208750e-03  5.72342426e-04 ...  1.21211004e-03
     6.62452634e-03 -1.42437574e-02]
   [-1.20792426e-02  1.66741142e-04  8.15137383e-03 ... -3.50654013e-02
    -9.46603343e-03 -3.82543020e-02]
   [ 1.58854406e-02  1.02695040e-02 -1.41241727e-02 ... -1.23617630e-02
    -5.41221444e-03 -4.35710326e-02]
   ...
   [-3.08039095e-02 -1.32695436e-02  3.28930318e-02 ... -1.90087892e-02
    -5.98782022e-03 -2.51824539e-02]
   [-1.26500512e-02 -7.94353429e-03 -2.97320783e-02 ... -8.77928641e-03
    -1.57757942e-02 -2.75921375e-02]
   [-3.31379063e-02  2.46990602e-02 -4.71759820e-03 ... -4.58475901e-03
     6.83892239e-03 -9.65156220e-03]]

  [[-7.67148042e-04 -1.65830832e-02 -1.19159017e-02 ...  6.24286849e-03
     7.60079874e-03 -1.80902183e-02]
   [ 2.49586739e-02  3.68976295e-02 -1.74194314e-02 ...  4.40805219e-02
     8.00087117e-03 -1.89169794e-02]
   [-2.05947994e-03 -1.78416520e-02 -9.46504436e-03 ... -8.06228083e-04
    -3.11954655e-02 -1.31438382e-03]
   ...
   

In [17]:
#print(WW2[1,0])

[[-0.01295047 -0.02532102 -0.01587163 -0.00400487  0.01152106 -0.03440093
   0.02510221]
 [-0.01594214  0.01365309  0.0207934  -0.00428279  0.01589915 -0.01013242
   0.01383109]
 [ 0.01786677  0.02972169  0.01891     0.01271127  0.00182282 -0.00797413
   0.01702197]
 [-0.0256801  -0.01688942 -0.00927839  0.01370859 -0.00602185  0.00812225
  -0.02140457]
 [ 0.00891152 -0.02243988  0.01167836  0.00298501 -0.00158755  0.02444264
   0.0034126 ]
 [ 0.01047056  0.0156278  -0.00605519 -0.01109279 -0.00485825 -0.00986225
   0.01158221]
 [-0.00159456  0.02342505 -0.00301597  0.00077849 -0.00484553 -0.02220511
  -0.02143711]]


## 卷积核的可视化(比较粗浅的实现方式,有待改进)
将卷积核的元素归一化到0-1之间,然后再相应放大到0-255之间

In [51]:
import cv2
#WW[0,0] -= WW[0,0].min()
#WW[0,0] /= WW[0,0].max()
#WW[0,0] = WW[0,0] * 255
for i in range(WW0.shape[0]):      #for循环可视化branch1.0的卷积核
    for j in range(WW0.shape[1]):
        WW0[i,j] -= WW0[i,j].min()
        WW0[i,j] /= WW0[i,j].max()
        WW0[i,j] = WW0[i,j] * 255
        cv2.imwrite('./Visualization generated/branch1.0/branch1.0_{0}_{1}.png'.format(i,j),WW0[i,j], [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

for i in range(WW2.shape[0]):      #for循环可视化
    for j in range(WW2.shape[1]):
        WW2[i,j] -= WW2[i,j].min()
        WW2[i,j] /= WW2[i,j].max()
        WW2[i,j] = WW2[i,j] * 255
        cv2.imwrite('./Visualization generated/branch1.2/branch1.2_{0}_{1}.png'.format(i,j),WW2[i,j], [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
for i in range(WW4.shape[0]):      #for循环可视化
    for j in range(WW4.shape[1]):
        WW4[i,j] -= WW4[i,j].min()
        WW4[i,j] /= WW4[i,j].max()
        WW4[i,j] = WW4[i,j] * 255
        cv2.imwrite('./Visualization generated/branch1.4/branch1.4_{0}_{1}.png'.format(i,j),WW4[i,j], [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
for i in range(WW5.shape[0]):      #for循环可视化
    for j in range(WW5.shape[1]):
        WW5[i,j] -= WW5[i,j].min()
        WW5[i,j] /= WW5[i,j].max()
        WW5[i,j] = WW5[i,j] * 255
        cv2.imwrite('./Visualization generated/branch1.5/branch1.5_{0}_{1}.png'.format(i,j),WW5[i,j], [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

In [52]:
branch1_0 = np.zeros([WW0.shape[2]*WW0.shape[0]*2, WW0.shape[2]*WW0.shape[1]*2])
#branch1_0 = branch1_0 * 255
for i in range(WW0.shape[0]):      
    for j in range(WW0.shape[1]):
        for m in range(WW0.shape[2]):
            for n in range(WW0.shape[3]):
                branch1_0[i*2*WW0.shape[2]+m,2*j*WW0.shape[2]+n] = WW0[i,j,m,n]
cv2.imwrite('./Visualization generated/branch1.0/branch1.0.png',branch1_0, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


branch1_2 = np.zeros([WW2.shape[2]*WW2.shape[0]*2, WW2.shape[2]*WW2.shape[1]*2])
#branch1_0 = branch1_0 * 255
for i in range(WW2.shape[0]):      
    for j in range(WW2.shape[1]):
        for m in range(WW2.shape[2]):
            for n in range(WW2.shape[3]):
                branch1_2[i*2*WW2.shape[2]+m,2*j*WW2.shape[2]+n] = WW2[i,j,m,n]
cv2.imwrite('./Visualization generated/branch1.2/branch1.2.png',branch1_2, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


branch1_4 = np.zeros([WW4.shape[2]*WW4.shape[0]*2, WW4.shape[2]*WW4.shape[1]*2])
#branch1_0 = branch1_0 * 255
for i in range(WW4.shape[0]):      
    for j in range(WW4.shape[1]):
        for m in range(WW4.shape[2]):
            for n in range(WW4.shape[3]):
                branch1_4[i*2*WW4.shape[2]+m,2*j*WW4.shape[2]+n] = WW4[i,j,m,n]
cv2.imwrite('./Visualization generated/branch1.4/branch1.4.png',branch1_4, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


branch1_5 = np.zeros([WW5.shape[2]*WW5.shape[0]*2, WW5.shape[2]*WW5.shape[1]*2])
#branch1_0 = branch1_0 * 255
for i in range(WW5.shape[0]):      
    for j in range(WW5.shape[1]):
        for m in range(WW5.shape[2]):
            for n in range(WW5.shape[3]):
                branch1_5[i*2*WW5.shape[2]+m,2*j*WW5.shape[2]+n] = WW5[i,j,m,n]
cv2.imwrite('./Visualization generated/branch1.5/branch1.5.png',branch1_5, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


True

## 特征图的可视化
思路:
1. 使用一张MCNN项目中的图片
2. 确定好想要可视化第几层的特征图,以及是哪一个卷积核卷积后的特征图
3. 使用conv2d定义中的forward()方法将图片往前传播,由于上面分析我们知道卷积核的矩阵为[X,1,D,D],其中X为卷积核个数,1代表单通道,D代表卷积核大小
4. 由于灰度图为二维,因此图片不能直接传入,需要经过preprocess_image()方法:使用mean和std对0-255的二维numpy数组进行处理,然后转化为float型的torch变量,然后使用unsqueeze_(0)方法将二维数组变成四维数组,才能传进forward中
5. 为了凸显出改特征图的特征,将第一次卷积后得到的特征图重新输入卷积神经网络,重复多次,可以看到特征越来越明显
6. 重新输入卷积神经网络的时候要用mean和std对数据进行与第4点相反的处理,再输入

In [14]:
import os
import numpy as np
import cv2
import torch
from torch.optim import Adam
from torchvision import models

from PIL import Image
import copy
import matplotlib.cm as mpl_color_map


from torch.autograd import Variable

#from misc_functions import preprocess_image, recreate_image, save_image
def save_image(im, path):

    if isinstance(im, np.ndarray):
        if len(im.shape) == 2:
            im = np.expand_dims(im, axis=0)
        if im.shape[0] == 1:
            #此时传进来的图像为3维的,其中第一维长度为1,可以看成图像的通道数,使用repeat拓展为3个通道,且每个通道的数据一样
            im = np.repeat(im, 3, axis=0)
            #经过上面处理,第一维为通道数,先对数组使用transpose进行倒转,再存储
        if im.shape[0] == 3:
            im = im.transpose(1, 2, 0) * 255
        im = Image.fromarray(im.astype(np.uint8))
    im.save(path)


def preprocess_image(pil_im, resize_im=False):

    mean = 0.485
    std = 0.229
    #使用mean和std对图像进行处理
    im_as_arr = np.float32(pil_im)
    im_as_arr /= 255
    im_as_arr -= mean 
    im_as_arr /= std
    # 转化为float tensor类型
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # 由于卷积核为四维,因此需要拓展成4维的tensor
    im_as_ten.unsqueeze_(0)
    im_as_ten.unsqueeze_(0)
    # 转为Variable变量,设置可进行求导操作
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var


def recreate_image(im_as_var):

    reverse_mean = -0.485
    reverse_std = 1/0.229
    #此时图像由4维变成3维
    recreated_im = copy.copy(im_as_var.data.numpy()[0])
    recreated_im /= reverse_std
    recreated_im -= reverse_mean
 #   recreated_im[recreated_im > 1] = 1
 #   recreated_im[recreated_im < 0] = 0
    recreated_im = np.round(recreated_im * 255)
    return recreated_im



class CNNLayerVisualization():
    
    def __init__(self, model, selected_layer, selected_filter,imgg):
        self.model = model
        self.model.eval()
        self.selected_layer = selected_layer
        self.selected_filter = selected_filter
        self.conv_output = 0
        self.created_image = imgg
      
        # 创建文件夹
        if not os.path.exists('./Visualization generated'):
            os.makedirs('./Visualization generated')


    def visualise_layer_without_hooks(self):
        # 将图像进行处理,转化为4维float tensor
        self.processed_image = preprocess_image(self.created_image)
        #定义一个优化器
        optimizer = Adam([self.processed_image], lr=0.1, weight_decay=1e-6)
        #重复输入31次MCNN网络
        for i in range(1, 31):
            optimizer.zero_grad()
            # x作为向前传播的图像
            x = self.processed_image
            x = x.cpu()
            x = x.cuda()
            #x = x.double()
            for index in range(self.selected_layer+1):
                x = self.model.DME.branch1[index].forward(x)            
            #选择特定的卷积核卷积的特征图
            self.conv_output = x[0, self.selected_filter]
            #loss函数使用特征图的平均值
            loss = -torch.mean(self.conv_output)
            print('Iteration:', str(i), 'Loss:', "{0:.2f}".format(loss.data.cpu().numpy()))
            # loss对图像像素求导
            loss.backward()
            # 更新图像像素
            optimizer.step()
            # 重新处理图像,以重新输入MCNN网络
            self.created_image = recreate_image(self.processed_image)
            # 保存图像
            if i % 5 == 0:
                if not os.path.exists('./Visualization generated/feature_branch1.{0}_filter{1}'.format(self.selected_layer, self.selected_filter)):
                        os.makedirs('./Visualization generated/feature_branch1.{0}_filter{1}'.format(self.selected_layer, self.selected_filter))
                im_path = './Visualization generated/feature_branch1.{0}_filter{1}/layer_vis_l'.format(self.selected_layer, self.selected_filter) + str(self.selected_layer) + '_f' + str(self.selected_filter) + '_iter' + str(i) + '.jpg'
                save_image(self.created_image, im_path)
                print("success")

#                 #使用这一部分,不需要多次迭代,把上面关于i的循环去掉
#                 feature = self.conv_output
#                 feature = feature.data.numpy()
#                 #sigmod归一化
#                 #feature= 1.0/(1+np.exp(-1*feature))
                
#                 #min-max归一化
#                 #maxnum = np.max(feature)
#                 #minnum = np.min(feature)
#                 #feature = (feature-minnum)/(maxnum-minnum)
#                 # to [0,255]
#                 feature=np.round(feature*255)
#                 feature = np.expand_dims(feature,axis=0)
#                 feature = np.repeat(feature,3,axis=0)
#                 feature = feature.transpose(1,2,0)
#                 print(feature.shape)
#                 cv2.imwrite(im_path,feature)


    
if __name__ == '__main__':
    #方便起见,可视化第一层的第一个卷积核卷积得到的特征图
    #若想可视化后面层数的特征图,加入for循环使用farward()得到
    #第0\2\4\5层为卷积层
    #可视化哪一个层?
    cnn_layer = 2
    #可视化哪一个卷积核?
    filter_pos = 6
    pretrained_model = net
    imgg = cv2.imread("2_5.jpg",0)#载入灰度图
    layer_vis = CNNLayerVisualization(pretrained_model, cnn_layer, filter_pos,imgg)

    layer_vis.visualise_layer_without_hooks()

('Iteration:', '1', 'Loss:', '-0.01')
('Iteration:', '2', 'Loss:', '-0.01')
('Iteration:', '3', 'Loss:', '-0.02')
('Iteration:', '4', 'Loss:', '-0.02')
('Iteration:', '5', 'Loss:', '-0.02')
success
('Iteration:', '6', 'Loss:', '-0.03')
('Iteration:', '7', 'Loss:', '-0.03')
('Iteration:', '8', 'Loss:', '-0.03')
('Iteration:', '9', 'Loss:', '-0.03')
('Iteration:', '10', 'Loss:', '-0.04')
success
('Iteration:', '11', 'Loss:', '-0.04')
('Iteration:', '12', 'Loss:', '-0.04')
('Iteration:', '13', 'Loss:', '-0.04')
('Iteration:', '14', 'Loss:', '-0.05')
('Iteration:', '15', 'Loss:', '-0.05')
success
('Iteration:', '16', 'Loss:', '-0.05')
('Iteration:', '17', 'Loss:', '-0.06')
('Iteration:', '18', 'Loss:', '-0.06')
('Iteration:', '19', 'Loss:', '-0.06')
('Iteration:', '20', 'Loss:', '-0.06')
success
('Iteration:', '21', 'Loss:', '-0.07')
('Iteration:', '22', 'Loss:', '-0.07')
('Iteration:', '23', 'Loss:', '-0.07')
('Iteration:', '24', 'Loss:', '-0.08')
('Iteration:', '25', 'Loss:', '-0.08')
su